In [ ]:
import flowmachine
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime

In [ ]:
flowmachine.connect()

In [ ]:
data_events_query = flowmachine.features.TotalLocationEvents(
    start="2016-01-01",
    stop="2016-01-07",
    table="events.mds",
    level="versioned-cell",
    interval="hour",
    size=10,
)

In [ ]:
data_events = data_events_query.get_dataframe()

In [ ]:
data_events.head(15)

In [ ]:
events_per_hour = data_events.groupby(["lon", "lat", "hour"]).sum().reset_index()

events_per_hour["weight"] = events_per_hour["total"]/(events_per_hour["total"].max())

hours = range(24)
data_for_heatmap = [events_per_hour[events_per_hour.hour==hour][["lat", "lon", "weight"]].values.tolist() for hour in hours]

In [ ]:
m = folium.Map(location=[27.96834547, 85.960067737], zoom_start=7, tiles='cartodbpositron')

HeatMapWithTime(data=data_for_heatmap, index=[str(hour) for hour in hours], radius=0.2, scale_radius=True).add_to(m)

m